In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dal-shemagh-detection-challenge/train_labels.csv
/kaggle/input/dal-shemagh-detection-challenge/labels/train/559.txt
/kaggle/input/dal-shemagh-detection-challenge/labels/train/557.txt
/kaggle/input/dal-shemagh-detection-challenge/labels/train/361.txt
/kaggle/input/dal-shemagh-detection-challenge/labels/train/40.txt
/kaggle/input/dal-shemagh-detection-challenge/labels/train/245.txt
/kaggle/input/dal-shemagh-detection-challenge/labels/train/431.txt
/kaggle/input/dal-shemagh-detection-challenge/labels/train/44.txt
/kaggle/input/dal-shemagh-detection-challenge/labels/train/574.txt
/kaggle/input/dal-shemagh-detection-challenge/labels/train/141.txt
/kaggle/input/dal-shemagh-detection-challenge/labels/train/487.txt
/kaggle/input/dal-shemagh-detection-challenge/labels/train/571.txt
/kaggle/input/dal-shemagh-detection-challenge/labels/train/372.txt
/kaggle/input/dal-shemagh-detection-challenge/labels/train/333.txt
/kaggle/input/dal-shemagh-detection-challenge/labels/train/540.txt
/

In [2]:
# Install Ultralytics (YOLO)
!pip install -q ultralytics
import os
import pandas as pd
import numpy as np
from ultralytics import YOLO
import yaml
from tqdm.notebook import tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.3 MB/s eta 0:00:0000:01
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# Define the dataset configuration
dataset_config = {
    'path': '/kaggle/input/dal-shemagh-detection-challenge', # Root dir
    'train': 'images/train',  # Train images (relative to path)
    'val': 'images/train',    # Using train as val for demo (Split this properly for real training!)
    'names': {
        0: 'person',
        1: 'shemagh'
    }
}

# Write to data.yaml
with open('data.yaml', 'w') as f:
    yaml.dump(dataset_config, f)

print("data.yaml created successfully!")

data.yaml created successfully!


In [4]:
# Load a pre-trained YOLO model (Recommended: yolo11s.pt or yolo11m.pt)
model = YOLO('yolo26n.pt') 

# Train the model
results = model.train(
    data='data.yaml',
    epochs=200,            # Increase this for better results!
    imgsz=640,
    batch=16,
    project='dal_challenge',
    name='shemagh_detector',
    exist_ok=True
)

Ultralytics 8.4.11 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=200, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo26n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=shemagh_detector, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspecti

KeyboardInterrupt: 

In [ ]:
import glob

# 1. Setup paths
test_images = glob.glob('/kaggle/input/dal-shemagh-detection-challenge/images/test/*.jpg')
submission_data = []

# 2. Define a function to calculate IoU (Intersection over Union)
def calculate_iou(box1, box2):
    """
    Calculate IoU between two boxes (x, y, w, h normalized).
    """
    # Convert from xywh to x1, y1, x2, y2
    b1_x1, b1_x2 = box1[0] - box1[2]/2, box1[0] + box1[2]/2
    b1_y1, b1_y2 = box1[1] - box1[3]/2, box1[1] + box1[3]/2
    
    b2_x1, b2_x2 = box2[0] - box2[2]/2, box2[0] + box2[2]/2
    b2_y1, b2_y2 = box2[1] - box2[3]/2, box2[1] + box2[3]/2

    # Intersection coordinates
    inter_x1 = max(b1_x1, b2_x1)
    inter_y1 = max(b1_y1, b2_y1)
    inter_x2 = min(b1_x2, b2_x2)
    inter_y2 = min(b1_y2, b2_y2)

    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    b1_area = (b1_x2 - b1_x1) * (b1_y2 - b1_y1)
    b2_area = (b2_x2 - b2_x1) * (b2_y2 - b2_y1)
    
    union_area = b1_area + b2_area - inter_area
    return inter_area / union_area if union_area > 0 else 0

# 3. Inference Loop
print(f"Processing {len(test_images)} images...")

for img_path in tqdm(test_images):
    filename = os.path.basename(img_path)
    
    # Run inference
    results = model.predict(img_path, conf=0.25, verbose=False)[0]
    
    prediction_strings = []
    person_boxes = []
    shemagh_boxes = []
    
    # Process detections
    if len(results.boxes) == 0:
        prediction_str = "-"
        right_place = 0
    else:
        for box in results.boxes:
            # Extract data
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            
            # YOLO returns xywh normalized if we access .xywhn
            x, y, w, h = box.xywhn[0].tolist()
            
            # Append to prediction string
            prediction_strings.append(f"{cls} {conf:.4f} {x:.4f} {y:.4f} {w:.4f} {h:.4f}")
            
            # Store boxes for logic check
            if cls == 0: # Assuming 0 is Person
                person_boxes.append([x, y, w, h])
            elif cls == 1: # Assuming 1 is Shemagh
                shemagh_boxes.append([x, y, w, h])
        
        prediction_str = " ".join(prediction_strings)
        
        # --- LOGIC FOR "RIGHT PLACE" ---
        # Heuristic: If we found at least one person and one shemagh
        # and they overlap (IoU > threshold), predict 1.
        
        right_place = 0 # Default to 0
        if person_boxes and shemagh_boxes:
            # Check every shemagh against every person
            for s_box in shemagh_boxes:
                for p_box in person_boxes:
                    iou = calculate_iou(s_box, p_box)
                    # Threshold: 0.1 means 10% overlap. Adjust this based on validation!
                    if iou > 0.1: 
                        right_place = 1
                        break
                if right_place == 1:
                    break
    
    submission_data.append([filename, right_place, prediction_str])


In [ ]:

# 4. Save to CSV
df_sub = pd.DataFrame(submission_data, columns=['filename', 'right_place', 'prediction_string'])
df_sub.to_csv('submission.csv', index=False)

print("Submission.csv generated successfully!")
print(df_sub.head())